# Index Performance Analysis

Refer to the [Introduction](/home/gnewy/workspace/starxetp/notebooks/introduction.ipynb) on how to use the notebook.

* Follow a similar procedure to that of [Portfolio Allocation and Pair Trading Strategy](https://blog.quantinsti.com/portfolio-allocation-pair-trading-strategy-python-project-ravindra-rawat/)
* [A strategy combining momentum and mean reversion](https://www.quantconnect.com/tutorials/strategy-library/combining-mean-reversion-and-momentum-in-forex-market)
* Using these factors we use regression to predict the returns of the coming month.

### ETP market quality
* Exchange Traded Product (ETP) investors would look at [five key metrics](https://www.ishares.com/us/insights/etf-trends/etp-market-quality-metrics) when assessing market quality. _We are only interested in the first three_.
   1. ___Usage___: liquidity of an ETP is a key component of market quality - ETP trading volumes are important because increased liquidity can create a network effect; i.e. the most heavily traded ETPs are typically the cheapest to trade, which spurs even more ___usage___.
   1. ___Tracking___: difference and volatility reflect an ETP's ability to deliver returns that are consistent its benchmark, as well as closely replicate benchmark performance consistently over time. An index ETP with high market quality should deliver this consistency in all market conditions.
   1. ___Trading costs___: When the cost of rebalancing an ETP higher than a tolerance band to that of the ETP’s underlying holdings, or exhibits less sensitivity to stressed market conditions, it is a potential signal of high market quality.
   1. ___Premium/discounts behavior___: ETP premiums and discounts in illiquid or volatile markets can indicate an ETP is providing price discovery—both signals of market quality.
   1. ___Primary market efficiency___ - A diverse set of authorized participants and a stable platform are crucial for insight into the ETP’s market qualityfor because the ETP’s primary market operations must be efficient.
* Preperations for the [Backtesting](https://teddykoker.com/2019/05/momentum-strategy-from-stocks-on-the-move-in-python/)


In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('default')     #switch betweeb 'default' and 'ignore'

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

# PREP THE DATA
* load the market cap data from the data files
* filter the data by the date range
* calculate the moving average

In [3]:
import sys
sys.path.insert(1, '../lib')
import clsDataETL as etl
import datetime

'''
    To filter data by a date range change the two date parameters below
'''
start_dt = datetime.date(2022,1,1)
end_dt = datetime.date(2022,3,1)

if debug:
    import importlib
    etl = importlib.reload(etl)

''' Set the data source and temporal range '''
_path = "../data/market_cap_2021-01-01_2022-06-01/"
# start_dt = datetime.date(2022,1,1)
# end_dt = datetime.date(2022,3,1)
''' Initialize the dataETL class '''
print("Loading and filtering data ... this may take a while.")
# clsETL = etl.ExtractLoadTransform(dataPath=_path, start_date=_start_dt, end_date=_end_dt)
clsETL = etl.ExtractLoadTransform()
''' Load data into dataframe '''
rec_marketcap_df=clsETL.load_data(dataPath=_path, start_date=start_dt, end_date=end_dt)
# rec_marketcap_df = clsETL.data
rec_marketcap_df.dropna(axis=0,how='any',subset='market_cap',inplace=True)
print("Loaded %d rows %s" % (rec_marketcap_df.shape[0],str(rec_marketcap_df.columns)))
# print(rec_marketcap_df.info(),"\n")
''' Transform data with coin ids in columns '''
piv_marketcap_df = rec_marketcap_df.pivot_table(values='market_cap', index=rec_marketcap_df.Date, columns='ID', aggfunc='first')
piv_marketcap_df.dropna(axis=1,how='all', inplace=True)
piv_marketcap_df.reset_index(inplace=True)
# print(piv_marketcap_df.info())
print("Data from %s to %s loaded and transformed into a pivot table with %d rows complete!" % (str(rec_marketcap_df.Date.min()),
                                                                    str(rec_marketcap_df.Date.max()),
                                                                    piv_marketcap_df.shape[0]))

All packages loaded successfully!
Loading and filtering data ... this may take a while.
Loaded 419 rows Index(['Date', 'ID', 'Symbol', 'market_cap'], dtype='object')
Data from 2022-01-01 to 2022-03-01 loaded and transformed into a pivot table with 60 rows complete!


In [3]:
from datetime import date

start_dt = date(2022,1,20)
end_dt = date(2022,2,10)

_cal_ops_dict = {
    "simp_move_avg" : "market_cap",
    "simp_move_std" : "market_cap",
    "simp_move_sum" : "market_cap",
    "momentum" : "market_cap",
}
_results_df = clsETL.get_rolling_measures(ticker_data=rec_marketcap_df,
                                                rolling_window_length=7,
                                                window_start_date = start_dt,
                                                window_end_date = end_dt,
                                                rolling_measure_dict = _cal_ops_dict,)
print(_results_df)


          Date        ID Symbol    market_cap  simp_move_avg_market_cap  \
19  2022-01-20   cardano    ada  4.312538e+10              4.312538e+10   
20  2022-01-21   cardano    ada  4.066241e+10              4.189389e+10   
21  2022-01-22   cardano    ada  3.636817e+10              4.005198e+10   
22  2022-01-23   cardano    ada  3.425647e+10              3.860311e+10   
23  2022-01-24   cardano    ada  3.595770e+10              3.807402e+10   
..         ...       ...    ...           ...                       ...   
36  2022-02-06  ethereum    eth  3.616004e+11                       NaN   
37  2022-02-07  ethereum    eth  3.660383e+11                       NaN   
38  2022-02-08  ethereum    eth  3.763510e+11                       NaN   
39  2022-02-09  ethereum    eth  3.740439e+11                       NaN   
40  2022-02-10  ethereum    eth  3.873212e+11                       NaN   

    simp_move_std_market_cap  simp_move_sum_market_cap  momentum_market_cap  
19                   

# COMPUTE RATE OF RETURNS

## Instantiate ETPreturns class

In [21]:
import sys
sys.path.insert(1, '../lib')
import clsETPreturns as returns

if debug:
    import importlib
    returns = importlib.reload(returns)

data_name = "coindesk"
clsROR = returns.RateOfReturns(name=data_name)
print(dir(clsROR))

All packages loaded successfully!
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'days_offset', 'get_coin_cov_cor_coef_matrix', 'get_geometric_return', 'get_holding_period_returns', 'get_logarithmic_returns', 'get_simple_returns', 'maximize_weights', 'name', 'p_val', 'sum_weighted_returns', 'window_length']


## Top-N Assets
${R(t,x_{i}) = log \left({R(t,x_{i}) \over R(t+1,x_{i})}\right) \le 0}$ implies that the ratio ${{R(t,x_{i}) \over R(t+1,x_{i})} \le 1.0}$. Thus there is an increase in the market cap value from ${t}$ to ${t+1}$. Therefore, the _top-N_ assets are the ones with the largest negative log ROR.  

In [22]:
import numpy as np

_kwargs = {'greater than': 0,
            'max num coins': 5}
#topN = 5
''' get the log rate of returns '''
actual_log_ror = clsROR.get_logarithmic_returns(rec_marketcap_df, value_col_name='market_cap')

_neg_log_df = actual_log_ror.copy()
_neg_log_df.dropna(axis=0, how='any', inplace=True)
_neg_log_df = _neg_log_df.sort_values(by=['Date','ror'])
_neg_log_df['ror'] = _neg_log_df['ror']*(-1)
#_topNassets_df = clsETL.get_fixed_topN_assets(_neg_log_df, N=topN, val_col_name='ror')
_topNassets_df = clsETL.get_significant_topN_assets(_neg_log_df,
                                                    val_col_name='ror', **_kwargs)
_topNassets_df['ror'] = _topNassets_df['ror']*(-1)
_topNassets_df=_topNassets_df.reindex()
print("Completed getting %d list with top assets" % (_topNassets_df.shape[0]))

Completed getting 164 list with top assets


## Risk measure
* expected return on an investment is the expected value of the probability distribution of possible returns it can provide. The purpose of calculating the expected return on an investment is to provide an investor with an idea of probable profit vs risk. This gives the investor a basis for comparison with the risk-free rate of return. The interest rate on 3-month U.S. Treasury bills is often used to represent the risk-free rate of return.
* Risk of a single asset is the standard devsion ${\sigma}$ for a given time. We calculated the T-day (e.g. 7-day) [moving standard deviation](https://www.danielstrading.com/education/technical-analysis-learning-center/moving-standard-deviation) ${\sigma}^{(T)}$
* MPT uses variance as its measure of risk

## Actual weighted returns SUM
Sum of the weighted actual portfolio allocation at time ${t}$: ${F_{t}({Y})}$  = $\sum_{k_i = 1}^{N<n} {w(t,x_{k_{i}})} \times {R(t,x_{k_i})}$

In [23]:
import pandas as pd

_size=100
_merged_actual_ror = pd.merge(_topNassets_df,
                              rec_marketcap_df,how='outer',
                              on=['Date','ID'])
_merged_actual_ror.dropna(axis=0, how='any', inplace=True)
_merged_actual_ror = _merged_actual_ror.sort_values(by=['Date','ror'], ascending=True)
print(_merged_actual_ror.head(10))

         Date        ID       ror Symbol    market_cap
0  2022-01-03    solana -0.008847    sol  5.460443e+10
1  2022-01-03   bitcoin -0.007786    btc  8.975361e+11
2  2022-01-04   cardano -0.039437    ada  4.257930e+10
3  2022-01-04    solana -0.031966    sol  5.288655e+10
4  2022-01-04    ripple -0.031305    xrp  3.966750e+10
5  2022-01-04  litecoin -0.020565    ltc  1.030904e+10
6  2022-01-04   bitcoin -0.019356    btc  8.803302e+11
7  2022-01-04  ethereum -0.018207    eth  4.486096e+11
8  2022-01-05    solana -0.011679    sol  5.227248e+10
9  2022-01-05    ripple -0.007475    xrp  3.937207e+10


### get weighted sum of returns metrics

In [24]:

_l_actual_weighted_sum = clsROR.sum_weighted_returns(
    _merged_actual_ror,
    size=_size,
    value_col_name='ror'
)
print(_l_actual_weighted_sum[:5])

[{'date': datetime.date(2022, 1, 3), 'coins': ['solana', 'bitcoin'], 'max_sum_row': 99, 'ror': [-0.008846735483933856, -0.0077864032143415245], 'best_weights': [0.0035272030547875907, 0.9964727969452124], 'weighted_ror_sum': -0.00779014322156192, 'weighted_market_cap_returns': 46431811980850.59}, {'date': datetime.date(2022, 1, 4), 'coins': ['cardano', 'solana', 'ripple', 'litecoin', 'bitcoin', 'ethereum'], 'max_sum_row': 5, 'ror': [-0.039437498732642046, -0.031965883608941766, -0.03130514703901118, -0.020565149890809785, -0.01935627635018719, -0.018207372396231266], 'best_weights': [0.002819115223337593, 0.018656217207850306, 7.735704773980446e-05, 0.23828507362589316, 0.06121008365499057, 0.6789521532401885], 'weighted_ror_sum': -0.019157065238421228, 'weighted_market_cap_returns': 25252445284525.95}, {'date': datetime.date(2022, 1, 5), 'coins': ['solana', 'ripple', 'litecoin', 'cardano', 'bitcoin'], 'max_sum_row': 69, 'ror': [-0.011679010839337992, -0.007475413472633437, -0.00595942

## Indicator based Trading strategies

### Momentum trading 
It assumes big moves will continue in the same direction. If the ETP weighted sum of returns is above a moving average, an uptrend is indicated. Directional Movement Index (DMI) is a technical indicator used by traders to help identify the strength of an uptrend or a downtrend in the market. The Average Direction Index (ADX) provides an indicator of the relative strength of the directional trend indicated by the Positive Direction Indicator (DI+) and the Negative Direction Indicator (DI-). Momentum trade can potentially reap big gains because getting in relatively early to a strong price trend is highly profitable.

### Mean reversion strategy
It allows traders to determine whether big moves will partly reverse or not. It assumes that the price of a stock always tends to move closer to the average price over time because most extreme events are often trailed by a period of normalization. Example - If the index droped 20% this month, the mean reversion theory would predict it will fall less than that percentage the following month. Use the [Relative Strength Index](https://phemex.com/academy/rsi-indicator-crypto-trading) (RSI) to complement their mean reversion strategy.
      * RSI is useful because it helps them determine which asset exhibits overbought or oversold price levels.
      * RSI Value >70% = Overbought, RSI Value <30% = Oversold
      * core of this indicator is based on the average upward market cap change vs. the average downward market cap change for a given period of time.
      * RSI = 100 – (100/1 + RS)
  We look at correlated assets to confirm the prediction. The quintessential mean reversion trading strategy has low-profit expectations and high frequency.


## Instantiate ETP Index class

In [48]:
import sys
sys.path.insert(1, '../lib')
import clsIndex as perform

if debug:
    import importlib
    perform = importlib.reload(perform)

data_name = "coindesk"
clsPerfIndex = perform.PortfolioPerformance(name=data_name)
print(dir(clsPerfIndex))

All packages loaded successfully!
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'get_adx', 'get_value_index', 'name', 'p_val', 'rebalance_etp', 'sharp_ratio', 'sortino_ratio']


## Indicator based portfolio selection
___Intent is to use the indicators to make decisions on portfolio asset selection and rebalancing using indicators___

## ADX Indicator
Average Directional Index (ADX) indicator determines the intensity or strength of a trend. It will ensure that we don't rebalance with a weaker trend because there is a high probability of reversal compared to a stronger trend. Hence, combining the rebalancing during a directional trades with a stronger trend will achieve higher [hit ratio](https://www.investopedia.com/terms/w/win-loss-ratio.asp) and higher ROR on the weighted ETP.

* ADX consists of three indicators that measure a trend’s strength and direction - Direction Movement Index (DMI): ADX line, DI+ line, and DI- line. 
* The [calculated ADX](https://www.investopedia.com/terms/w/wilders-dmi-adx.asp#:~:text=The%20DMI%20is%20a%20collection,25%20indicates%20a%20strong%20trend.) value above 25 indicates that the trend is relatively strong and a value below 20 indicates that the trend is weak or that the markets are trading sideways
   * When the DI+ line is on top of DI-, the markets are in a bullish trend. Conversely, when the DI- line is above the DI+ line, the markets are in a bearish trend.
   * The ADX indicator helps traders calculate the expansion or contraction of an asset’s price range for a specific time.
* The DMI and ADX values are determined based on the range of price movements during the last 14 trading periods.
* Follow a similar process discussed in [Mathematical Intuition of the ADX Indicator: A Python Approach](https://blog.quantinsti.com/adx-indicator-python/).

In [49]:
''' Create ADX relevant columns '''
if debug:
    import importlib
    etl = importlib.reload(etl)
    
''' ADX function is defined in clsIndex.get_adx() It will return 
    a time series dataframe with daily ADX, +DI, and -DI values '''
''' Set the data source and temporal range '''
_start_dt = rec_marketcap_df.Date.min()
_end_dt = rec_marketcap_df.Date.max()
adx_df = clsPerfIndex.get_adx(ticker_data=rec_marketcap_df,
                            rolling_window_length=7,
                            value_col_name='market_cap',
                            window_start_date = _start_dt,
                            window_end_date = _end_dt,
                            )
print(adx_df.head(5))
# print(adx_df.info())

All packages loaded successfully!
Index(['Date', 'ID', 'Symbol', 'market_cap'], dtype='object')
All packages loaded successfully!
{'simp_move_sum': '+DM', 'simp_move_avg': '+DM'}
{'simp_move_sum': '-DM', 'simp_move_avg': '-DM'}
          Date        ID Symbol    market_cap       ror       +DM       -DM  \
45  2022-02-15  ethereum    eth  3.513689e+11  0.017559  0.000000  0.017559   
46  2022-02-16  ethereum    eth  3.817911e+11  0.083037  0.000000  0.083037   
47  2022-02-17  ethereum    eth  3.760719e+11 -0.015093  0.015093  0.000000   
48  2022-02-18  ethereum    eth  3.452526e+11 -0.085504  0.085504  0.000000   
49  2022-02-19  ethereum    eth  3.339976e+11 -0.033143  0.033143  0.000000   

    simp_move_sum_+DM  simp_move_avg_+DM  simp_move_sum_-DM  \
45           0.000000                0.0           0.017559   
46           0.000000                0.0           0.100596   
47           0.015093                0.0           0.100596   
48           0.100597                0.0     

In [50]:
# for col in adx_df.filter(items=['+DM','-DM']).columns:
#     adx_df['shift_'+col]=adx_df[col].shift(1, axis = 0)
# adx_df[adx_df.filter(like='DM').columns]=adx_df[adx_df.filter(like='DM').columns].fillna(value=0)
# adx_df['smooth+DM']=(adx_df['simp_move_sum_+DM'].subtract(adx_df['simp_move_avg_+DM'])).add(adx_df['shift_+DM'])
# adx_df['smooth-DM']=(adx_df['simp_move_sum_-DM'].subtract(adx_df['simp_move_avg_-DM'])).add(adx_df['shift_-DM'])
# adx_df['+DI']=adx_df['simp_move_avg_+DM'].div(adx_df['smooth+DM'])
# adx_df['-DI']=adx_df['simp_move_avg_-DM'].div(adx_df['smooth-DM'])
print(adx_df.head(5))
adx_df.to_csv("../data/adx.csv")

          Date        ID Symbol    market_cap       ror       +DM       -DM  \
45  2022-02-15  ethereum    eth  3.513689e+11  0.017559  0.000000  0.017559   
46  2022-02-16  ethereum    eth  3.817911e+11  0.083037  0.000000  0.083037   
47  2022-02-17  ethereum    eth  3.760719e+11 -0.015093  0.015093  0.000000   
48  2022-02-18  ethereum    eth  3.452526e+11 -0.085504  0.085504  0.000000   
49  2022-02-19  ethereum    eth  3.339976e+11 -0.033143  0.033143  0.000000   

    simp_move_sum_+DM  simp_move_avg_+DM  simp_move_sum_-DM  \
45           0.000000                0.0           0.017559   
46           0.000000                0.0           0.100596   
47           0.015093                0.0           0.100596   
48           0.100597                0.0           0.100596   
49           0.133740                0.0           0.100596   

    simp_move_avg_-DM  shift_+DM  shift_-DM  smooth+DM  smooth-DM  +DI  -DI  
45                0.0   0.000000   0.000000   0.000000   0.017559  N

## Index based optimization
___The objective is to optimize the performance ratios for an ETP portfolio asset picks and rebalancing___

### Performance Ratios
The calculation is adjusted for an ETP for a given time period ${\left[{T_{min},T_{max}}\right]}$.
* For an ETP portfolio consisting top ${N < n}$ set of assets ${Y}$
    * the weighted ROR at time ${t}$ is ${F({t,Y})}$  = $\sum_{k_i = 1}^{N<n} {w(t,x_{k_{i}})} \times {log \left({R(t,x_{k_i}) \over R(t+1,x_{k_i})}\right)}$
* Let ${z}$ be the asset (e.g. Bitcoin) that sets the benchmark to offer a risk free ROR.
    * Similarly to the Y set of assets, the weighted ROR for ${z}$ at time ${t}$ is ${F({t,z})}$  = ${w(t,z)} \times {log \left({R(t,z)} \over R(t+1,z)\right)}$

### Sharpe Ratio
It measures the performance of an investment (e.g., security or portfolio) compared to a risk-free asset, after adjusting for its risk. It is poor at estimating tail risks and as a results gave rise to the [PMPT](https://www.investopedia.com/terms/p/pmpt.asp).
* ${\forall_{t \in {\left[{T_{min},T_{max}}\right]}}}$
    * ___Sharpe ration___ = ${{\mu \left({F(t,x_{k_i})}\right) - \mu \left({F(t,z)}\right)} \over {\sigma(F(t,x_{k_i}))} }$; where ${\mu}$ is the expected vallue and ${\sigma}$ is the standard deviation

In [31]:
sharp = clsPerfIndex.sharp_ratio(piv_marketcap_df, investment=100, risk_free_rate=0.01/365)
print("\n Sharp Ratio")
print(sharp.sort_values(ascending=False))

-0.0004980844352952042

 Sharp Ratio
ID
ripple          0.016981
bitcoin         0.000000
ethereum       -0.051617
litecoin       -0.060197
cardano        -0.075909
solana         -0.104899
bitcoin_cash         NaN
dtype: float64


### Sortino Ratio
It is a portfolio optimization methodology that uses the downside risk of returns instead of the mean variance of investment returns used by the [MPT](https://blog.quantinsti.com/modern-portfolio-capital-asset-pricing-fama-french-three-factor-model/)
* ${\forall_{t \in {\left[{T_{min},T_{max}}\right]}}}$
    * ___Sortino ratio___ is almost the same as the Sharpe ratio except that the ${\sigma^-(F(t,x_{k_i}))}$ is the downside standard deviation of returns 

In [32]:
sortino = clsPerfIndex.sortino_ratio(piv_marketcap_df, investment=100, risk_free_rate=0.01/365)
print("\n Sortino Ratio")
print(sortino.sort_values(ascending=False))


 Sortino Ratio
ID
ripple          0.030394
bitcoin         0.000000
ethereum       -0.080442
litecoin       -0.094637
cardano        -0.149946
solana         -0.169963
bitcoin_cash         NaN
dtype: float64


### Calmar ratio
The Calmar ratio is the average annual rate of return for the last 36 months divided by the maximum drawdown for the last 36 months. It is calculated on a monthly basis. The Calmar ratio changes gradually and serves to smooth out the overachievement and underachievement periods of a performance more readily than the Sharpe ratio.

## Value Index

In [33]:
import plotly.express as px

index_df = clsPerfIndex.get_value_index(piv_marketcap_df)

_min_date = (index_df["Date"].min()).date()
_max_date = (index_df["Date"].max()).date()
_title = "Asset class value index "+str(_min_date)+" to "+str(_max_date)
fig = px.line(index_df, x="Date", y=index_df.columns,
              hover_data={"Date": "|%B %d, %Y"},
              title=_title)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

KeyboardInterrupt: 